In [14]:
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers.core import Dense, Activation, Dropout

import pandas as pd
import numpy as np

# Read data
train = pd.read_csv('input/train.csv')
labels = train.iloc[:,0].values.astype('int32')
X_train = (train.iloc[:,1:].values).astype('float32')
X_test = (pd.read_csv('input/test.csv').values).astype('float32')


In [15]:
# convert list of labels to binary class matrix
y_train = np_utils.to_categorical(labels) 

# pre-processing: divide by max and substract mean
scale = np.max(X_train)
X_train /= scale
X_test /= scale

mean = np.std(X_train)
X_train -= mean
X_test -= mean

input_dim = X_train.shape[1]
nb_classes = y_train.shape[1]

# Here's a Deep Dumb MLP (DDMLP)
model = Sequential()
model.add(Dense(128, input_dim=input_dim))
model.add(Activation('relu'))
model.add(Dropout(0.15))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.15))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.15))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

# we'll use categorical xent for the loss, and RMSprop as the optimizer
model.compile(loss='categorical_crossentropy', optimizer='rmsprop',metrics=['accuracy'])

In [16]:
print("Training...")
model.fit(X_train, y_train, epochs=20, batch_size=16, validation_split=0.1,  verbose=2)

Training...
Train on 37800 samples, validate on 4200 samples
Epoch 1/20
11s - loss: 0.4063 - acc: 0.8748 - val_loss: 0.1797 - val_acc: 0.9483
Epoch 2/20
9s - loss: 0.2415 - acc: 0.9330 - val_loss: 0.2200 - val_acc: 0.9419
Epoch 3/20
14s - loss: 0.2440 - acc: 0.9407 - val_loss: 0.1617 - val_acc: 0.9583
Epoch 4/20
12s - loss: 0.2470 - acc: 0.9437 - val_loss: 0.2292 - val_acc: 0.9510
Epoch 5/20
10s - loss: 0.2574 - acc: 0.9456 - val_loss: 0.2304 - val_acc: 0.9548
Epoch 6/20
9s - loss: 0.2724 - acc: 0.9474 - val_loss: 0.1997 - val_acc: 0.9617
Epoch 7/20
9s - loss: 0.2836 - acc: 0.9482 - val_loss: 0.2665 - val_acc: 0.9619
Epoch 8/20
10s - loss: 0.2755 - acc: 0.9511 - val_loss: 0.2658 - val_acc: 0.9560
Epoch 9/20
9s - loss: 0.2909 - acc: 0.9497 - val_loss: 0.3229 - val_acc: 0.9586
Epoch 10/20
9s - loss: 0.2914 - acc: 0.9526 - val_loss: 0.4589 - val_acc: 0.9471
Epoch 11/20
9s - loss: 0.3056 - acc: 0.9520 - val_loss: 0.2871 - val_acc: 0.9588
Epoch 12/20
9s - loss: 0.3085 - acc: 0.9534 - val_lo

In [6]:
print("Generating test predictions...")
preds = model.predict_classes(X_test, verbose=0)

def write_preds(preds, fname):
    pd.DataFrame({"ImageId": list(range(1,len(preds)+1)), "Label": preds}).to_csv(fname, index=False, header=True)

write_preds(preds, "keras-mlp.csv")

Generating test predictions...


In [7]:
test_labels = X_test.ix[:,0].values.astype('int32')